In [7]:
%config IPCompleter.use_jedi=False
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

from Utils import print_memory_usage, frequency_encode_stations, evaluate_model, train, validate, load_data 

path_train = "Data/Train.parquet"
path_val = "Data/Validation.parquet"

# Logistic Regression

The first model we try is a simple logistic regression model.

## Variable Encodings

The first question is how to choose a proper encoding for the features:

1. Hour of the day: One approach is convert it into a cyclic variable. Another is to use fixed "binned" time intervales like morning, midday, evening.
2. Station ID: Since an ordinal encoding does not make sense, we will try a frequency encoding.
2. Numerical Features should be scaled to be comparable, since we use weight decay which depends on the scale on the coefficients.

In [8]:
def print_model_coefficients(model,feature_names):
    coefficients = pd.Series([model.intercept_[0]]+list(model.coef_[0]),index=["intercept"]+list(feature_names))
    print("Coefficients: ")
    print(coefficients)

In [13]:
def hour_to_coordinate(data,features):
    h = data["starttime"].dt.hour
    xh = np.sin(2*np.pi*(h)/24)
    yh = np.cos(2*np.pi*(h)/24)
    data["xh"] = xh
    data["yh"] = yh
    features = features + ["xh","yh"]
    return data,features

In [18]:
def categorize_tod(data,features,add_interactions=False):
    #Categories based on plots in Analysis Notebook
    hours = data["starttime"].dt.hour 
    bins=[-1,6,10,15,24] 
    names=[0,1,2,3]
    tod = pd.cut(hours,bins,labels=names)
    tod = pd.get_dummies(tod,prefix="tod",drop_first=True)
    new=[tod]
    features = features + list(tod.columns)
    if add_interactions:
        interaction = tod.mul(data["business day"],axis=0)
        interaction.columns = ["business x " + c for c in tod.columns]
        new += [interaction]
        features = features + list(interaction.columns)
    data = data.join(new) 
    return data,features
def categorize_tod_interactions(data,features):
    """Wrapper for convenience."""
    return categorize_tod(data,features,add_interactions=True)

Now lets try logistic regression with different features and preprocessings.

In [11]:
def add_hours(data,features):
    features = features + ["hour"]
    data["hour"] = data["starttime"].dt.hour
    return data,features

In [12]:
pre = add_hours
features=["tripduration", "summer","business day", "haversine distance", "is_roundtrip", "speed"]
scaler = MinMaxScaler()
features_to_scale = ["tripduration","haversine distance", "speed"]
clf = LogisticRegression(max_iter=100)
print("Training: ")
clf,feature_names = train(path_train,clf,features,preprocess=pre,scaler=scaler,features_to_scale=features_to_scale,fit_scaler = True)
print("Validation: ")
validate(clf,path_val,features,preprocess=pre,scaler=scaler,features_to_scale=features_to_scale,fit_scaler=False);

Training: 
Accuracy: 0.8941807341401965
Confusion: 
[[0.9817782  0.0182218 ]
 [0.81854074 0.18145926]]
MCC: 0.2732489467624145
Validation: 
Accuracy: 0.8942341632136787
Confusion: 
[[0.98194031 0.01805969]
 [0.81980536 0.18019464]]
MCC: 0.2724035976893507


Categorizing tod instead slightly improves the model

In [9]:
pre = categorize_tod
features=["tripduration", "summer","business day", "haversine distance", "is_roundtrip", "speed"]
scaler = MinMaxScaler()
features_to_scale = ["tripduration","haversine distance", "speed"]
clf = LogisticRegression(max_iter=100)
print("Training: ")
clf,feature_names = train(path_train,clf,features,preprocess=pre,scaler=scaler,features_to_scale=features_to_scale,fit_scaler = True)
print("Validation: ")
validate(clf,path_val,features,preprocess=pre,scaler=scaler,features_to_scale=features_to_scale,fit_scaler=False);

Training: 
Accuracy: 0.895168003048048
Confusion: 
[[0.98156348 0.01843652]
 [0.8077737  0.1922263 ]]
MCC: 0.28574764216388404
Validation: 
Accuracy: 0.8951586467528946
Confusion: 
[[0.98161271 0.01838729]
 [0.80868733 0.19131267]]
MCC: 0.2848225615211647


Alternatively, we can try a cyclic encoding, but the performance does not change much.

In [14]:
pre = hour_to_coordinate
features=["tripduration", "summer","business day", "haversine distance", "is_roundtrip", "speed"]
scaler = MinMaxScaler()
features_to_scale = ["tripduration","haversine distance", "speed"]
clf = LogisticRegression(max_iter=100)
print("Training: ")
clf,feature_names = train(path_train,clf,features,preprocess=pre,scaler=scaler,features_to_scale=features_to_scale,fit_scaler = True)
print("Validation: ")
validate(clf,path_val,features,preprocess=pre,scaler=scaler,features_to_scale=features_to_scale,fit_scaler=False);

Training: 
Accuracy: 0.8951237382751984
Confusion: 
[[0.98170481 0.01829519]
 [0.80932799 0.19067201]]
MCC: 0.28450710970717225
Validation: 
Accuracy: 0.8951780975204277
Confusion: 
[[0.98180349 0.01819651]
 [0.8100627  0.1899373 ]]
MCC: 0.28404669400888083


We can also add an interaction term between tod and business day, since we have seen that the distribution of subscribers over the day depends strongly on whether its a business day or holiday. However, the effect on model performance is minimal.

In [15]:
pre = categorize_tod_interactions
features=["tripduration", "summer","business day", "haversine distance", "is_roundtrip", "speed"]
scaler = MinMaxScaler()
features_to_scale = ["tripduration","haversine distance", "speed"]
clf = LogisticRegression(max_iter=100)
print("Training: ")
clf,feature_names = train(path_train,clf,features,preprocess=pre,scaler=scaler,features_to_scale=features_to_scale,fit_scaler = True)
print("Validation: ")
validate(clf,path_val,features,preprocess=pre,scaler=scaler,features_to_scale=features_to_scale,fit_scaler=False);

Training: 
Accuracy: 0.8953738092132513
Confusion: 
[[0.98178976 0.01821024]
 [0.8077345  0.1922655 ]]
MCC: 0.28691115450030363
Validation: 
Accuracy: 0.8953239782769251
Confusion: 
[[0.98179514 0.01820486]
 [0.80866118 0.19133882]]
MCC: 0.2857544379373309


Lets also add station ids. We try it once without any encoding.

In [8]:
pre = categorize_tod_interactions
features=["tripduration", "summer","business day", "haversine distance", "is_roundtrip", "speed", "start station id", "end station id"]
scaler = MinMaxScaler()
features_to_scale = ["tripduration","haversine distance", "speed"]
clf = LogisticRegression(max_iter=100)
print("Training: ")
clf,feature_names = train(path_train,clf,features,preprocess=pre,scaler=scaler,features_to_scale=features_to_scale,fit_scaler = True)
print("Validation: ")
validate(clf,path_val,features,preprocess=pre,scaler=scaler,features_to_scale=features_to_scale,fit_scaler=False);

Training: 


/home/thomas/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy: 0.8907051986151061
Confusion: 
[[0.99827718 0.00172282]
 [0.98453545 0.01546455]]
MCC: 0.07564733748703922
Validation: 
Accuracy: 0.8907370296275235
Confusion: 
[[0.99829263 0.00170737]
 [0.98490229 0.01509771]]
MCC: 0.0743261364445043


Using unencoded station ids does not make sense for logistic regression because the ids do not reflect an ordering.
Using one-hot encoding might work, but requires a lot of memory and introduces many variables (one per station ids, so about 800). Therefore, we will use a frequency encoding, mapping the stations to a corresponding fraction of customers / subscribers in the training set.

In [10]:
def preprocess_1(data,features):
    data, features = categorize_tod(data,features,add_interactions=True)
    data, features = frequency_encode_stations(data,features)
    return data,features

In [11]:
pre = preprocess_1
features=["tripduration", "summer","business day", "haversine distance", "is_roundtrip", "speed"]
scaler = MinMaxScaler()
features_to_scale = ["tripduration","haversine distance", "speed", "start customer freq","start subscriber freq","stop customer freq","stop subscriber freq"]
clf = LogisticRegression(max_iter=100)
print("Training: ")
clf,feature_names = train(path_train,clf,features,preprocess=pre,scaler=scaler,features_to_scale=features_to_scale,fit_scaler = True)
print("Validation: ")
validate(clf,path_val,features,preprocess=pre,scaler=scaler,features_to_scale=features_to_scale,fit_scaler=False);

Training: 
Accuracy: 0.90196339653685
Confusion: 
[[0.9819877  0.0180123 ]
 [0.74914037 0.25085963]]
MCC: 0.35639811832864116
Validation: 
Accuracy: 0.9019698478778354
Confusion: 
[[0.98198784 0.01801216]
 [0.74947835 0.25052165]]
MCC: 0.355995401986027


This improves the model alot.
Two more things I want to try: Using balanced labels, and adding start-end station interactions:

In [14]:
pre = preprocess_1
features=["tripduration", "summer","business day", "haversine distance", "is_roundtrip", "speed"]
scaler = MinMaxScaler()
features_to_scale = ["tripduration","haversine distance", "speed", "start customer freq","start subscriber freq","stop customer freq","stop subscriber freq"]
clf = LogisticRegression(max_iter=300,class_weight="balanced")
print("Training: ")
clf,feature_names = train(path_train,clf,features,preprocess=pre,scaler=scaler,features_to_scale=features_to_scale,fit_scaler = True)
print("Validation: ")
validate(clf,path_val,features,preprocess=pre,scaler=scaler,features_to_scale=features_to_scale,fit_scaler=False);

Training: 
Accuracy: 0.7933540912949856
Confusion: 
[[0.79473631 0.20526369]
 [0.2178921  0.7821079 ]]
MCC: 0.40641754869466673
Validation: 
Accuracy: 0.7927966942848497
Confusion: 
[[0.79400006 0.20599994]
 [0.21700022 0.78299978]]
MCC: 0.40608697111151654


And now with start-end station interactions

In [7]:
def preprocess_2(data,features):
    data, features = categorize_tod(data,features,add_interactions=True)
    data, features = frequency_encode_stations(data,features)
    data["customer start x stop"] = data["start customer freq"].mul(data["stop customer freq"],axis=0)
    data["subscriber start x stop"] = data["start subscriber freq"].mul(data["stop subscriber freq"],axis=0)
    features = features+["customer start x stop", "subscriber start x stop"]
    return data,features

In [9]:
pre = preprocess_2
features=["tripduration", "summer","business day", "haversine distance", "is_roundtrip", "speed"]
scaler = MinMaxScaler()
features_to_scale = ["tripduration","haversine distance", "speed", "start customer freq","start subscriber freq","stop customer freq","stop subscriber freq", "customer start x stop", "subscriber start x stop"]
clf = LogisticRegression(max_iter=300,class_weight="balanced")
print("Training: ")
clf,feature_names = train(path_train,clf,features,preprocess=pre,scaler=scaler,features_to_scale=features_to_scale,fit_scaler = True)
print("Validation: ")
validate(clf,path_val,features,preprocess=pre,scaler=scaler,features_to_scale=features_to_scale,fit_scaler=False);

Training: 
Accuracy: 0.7953510832927155
Confusion: 
[[0.79695971 0.20304029]
 [0.21773726 0.78226274]]
MCC: 0.4090453628418705
Validation: 
Accuracy: 0.7946691167006006
Confusion: 
[[0.79614744 0.20385256]
 [0.21736629 0.78263371]]
MCC: 0.40828140331115814


Finally, lets have a look at the model coefficients to see which features are most important.

In [10]:
print_model_coefficients(clf,feature_names)

Coefficients: 
intercept                   1.830502
tripduration                4.468506
is_roundtrip               -1.225583
haversine distance          6.581761
speed                     -14.932430
business day               -0.698999
summer                      0.661630
tod_1                      -0.036107
tod_2                       0.203431
tod_3                       0.121509
business x tod_1           -0.512030
business x tod_2            0.123386
business x tod_3           -0.267613
start customer freq         3.717062
start subscriber freq      -1.707202
stop customer freq          3.717062
stop subscriber freq       -1.707202
customer start x stop      -5.545453
subscriber start x stop     1.808514
dtype: float64
